In [2]:
import pandas as pd 
import os 

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import seaborn as sns

from datasets import load_dataset
import numpy as np

# Utils

In [44]:
# Copyright 2024 Bytedance Ltd. and/or its affiliates
# Copyright 2022 EleutherAI and the HuggingFace Inc. team. All rights reserved.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Adapted from https://github.com/EleutherAI/lm-evaluation-harness/blob/main/lm_eval/tasks/hendrycks_math/utils.py


def compute_score(solution_str, ground_truth) -> float:
    retval = 0.
    try:
        solution_string_in_last_boxed = last_boxed_only_string(solution_str)
        ground_truth_string_in_last_boxed = last_boxed_only_string(ground_truth)

        if solution_string_in_last_boxed is not None:
            answer = remove_boxed(solution_string_in_last_boxed)
        if ground_truth_string_in_last_boxed is not None:
            ground_truth = remove_boxed(ground_truth_string_in_last_boxed)
            
        if is_equiv(answer, ground_truth):
            retval = 1.
    except Exception as e:
        print(e)

    return retval


# string normalization from https://github.com/EleutherAI/lm-evaluation-harness/blob/master/lm_eval/tasks/hendrycks_math.py
def is_equiv(str1, str2, verbose=False):
    if str1 is None and str2 is None:
        print("WARNING: Both None")
        return True
    if str1 is None or str2 is None:
        return False

    try:
        ss1 = strip_string(str1)
        ss2 = strip_string(str2)
        if verbose:
            print(ss1, ss2)
        return ss1 == ss2
    except Exception:
        return str1 == str2


def remove_boxed(s):
    if "\\boxed " in s:
        left = "\\boxed "
        assert s[:len(left)] == left
        return s[len(left):]

    left = "\\boxed{"

    assert s[:len(left)] == left
    assert s[-1] == "}"

    return s[len(left):-1]


def last_boxed_only_string(string):
    idx = string.rfind("\\boxed")
    if "\\boxed " in string:
        return "\\boxed " + string.split("\\boxed ")[-1].split("$")[0]
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return None

    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1

    if right_brace_idx is None:
        retval = None
    else:
        retval = string[idx:right_brace_idx + 1]

    return retval


def fix_fracs(string):
    substrs = string.split("\\frac")
    new_str = substrs[0]
    if len(substrs) > 1:
        substrs = substrs[1:]
        for substr in substrs:
            new_str += "\\frac"
            if substr[0] == "{":
                new_str += substr
            else:
                try:
                    assert len(substr) >= 2
                except AssertionError:
                    return string
                a = substr[0]
                b = substr[1]
                if b != "{":
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}{" + b + "}" + post_substr
                    else:
                        new_str += "{" + a + "}{" + b + "}"
                else:
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}" + b + post_substr
                    else:
                        new_str += "{" + a + "}" + b
    string = new_str
    return string


def fix_a_slash_b(string):
    if len(string.split("/")) != 2:
        return string
    a = string.split("/")[0]
    b = string.split("/")[1]
    try:
        a = int(a)
        b = int(b)
        assert string == "{}/{}".format(a, b)
        new_string = "\\frac{" + str(a) + "}{" + str(b) + "}"
        return new_string
    except AssertionError:
        return string


def remove_right_units(string):
    # "\\text{ " only ever occurs (at least in the val set) when describing units
    if "\\text{ " in string:
        splits = string.split("\\text{ ")
        assert len(splits) == 2
        return splits[0]
    else:
        return string


def fix_sqrt(string):
    if "\\sqrt" not in string:
        return string
    splits = string.split("\\sqrt")
    new_string = splits[0]
    for split in splits[1:]:
        if split[0] != "{":
            a = split[0]
            new_substr = "\\sqrt{" + a + "}" + split[1:]
        else:
            new_substr = "\\sqrt" + split
        new_string += new_substr
    return new_string


def strip_string(string):
    # linebreaks
    string = string.replace("\n", "")

    # remove inverse spaces
    string = string.replace("\\!", "")

    # replace \\ with \
    string = string.replace("\\\\", "\\")

    # replace tfrac and dfrac with frac
    string = string.replace("tfrac", "frac")
    string = string.replace("dfrac", "frac")

    # remove \left and \right
    string = string.replace("\\left", "")
    string = string.replace("\\right", "")

    # Remove circ (degrees)
    string = string.replace("^{\\circ}", "")
    string = string.replace("^\\circ", "")

    # remove dollar signs
    string = string.replace("\\$", "")

    # remove units (on the right)
    string = remove_right_units(string)

    # remove percentage
    string = string.replace("\\%", "")
    string = string.replace("\%", "")  # noqa: W605

    # " 0." equivalent to " ." and "{0." equivalent to "{." Alternatively, add "0" if "." is the start of the string
    string = string.replace(" .", " 0.")
    string = string.replace("{.", "{0.")
    # if empty, return empty string
    if len(string) == 0:
        return string
    if string[0] == ".":
        string = "0" + string

    # to consider: get rid of e.g. "k = " or "q = " at beginning
    if len(string.split("=")) == 2:
        if len(string.split("=")[0]) <= 2:
            string = string.split("=")[1]

    # fix sqrt3 --> sqrt{3}
    string = fix_sqrt(string)

    # remove spaces
    string = string.replace(" ", "")

    # \frac1b or \frac12 --> \frac{1}{b} and \frac{1}{2}, etc. Even works with \frac1{72} (but not \frac{72}1). Also does a/b --> \\frac{a}{b}
    string = fix_fracs(string)

    # manually change 0.5 --> \frac{1}{2}
    if string == "0.5":
        string = "\\frac{1}{2}"

    # NOTE: X/Y changed to \frac{X}{Y} in dataset, but in simple cases fix in case the model output is X/Y
    string = fix_a_slash_b(string)

    return string


<>:200: SyntaxWarning: invalid escape sequence '\%'
<>:200: SyntaxWarning: invalid escape sequence '\%'
/home/al2644/tmp/ipykernel_3880409/3752985593.py:200: SyntaxWarning: invalid escape sequence '\%'
  string = string.replace("\%", "")  # noqa: W605


# Reasoning Results

In [8]:
root = "/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/distraction/"
fname = "Qwen2.5-Math-1.5B-DrGRPO-Zero.pickle"
df = pd.read_pickle(os.path.join(root, fname))

In [26]:
no_response = df[(df['original_correct'] == 0.0) & (df['distractor_correct'] == 0.0)]
response = df[(df['original_correct'] == 0.0) & (df['distractor_correct'] == 0.0) & (df['post_distraction_response'] != '')]

In [49]:
response['post_distraction_response'].iloc[10]

" But we need to check if there are any other values of \\(n\\) that can give us \\(\\gcd(n + 5, n + 11) = 3\\). Since \\(n + 5\\) and \\(n + 11\\) differ by 6, the only way for their gcd to be 3 is if they are both multiples of 3 but not multiples of 9. This means \\(n \\equiv 1 \\pmod{3}\\) and \\(n \\not\\equiv 0 \\pmod{3}\\). So, \\(n\\) can be any number of the form \\(3k + 1\\), where \\(k\\) is an integer. For example, \\(n = 1, 4, 7, 10, \\ldots\\).\n\n4. **Check if \\(d = 6\\) is possible:**\n   If \\(d = 6\\), then both \\(n + 5\\) and \\(n + 11\\) must be divisible by 6. This means \\(n \\equiv -5 \\equiv 1 \\pmod{6}\\) and \\(n \\equiv -11 \\equiv 1 \\pmod{6}\\). So, \\(n \\equiv 1 \\pmod{6}\\). For example, if \\(n = 1\\), then \\(n + 5 = 6\\) and \\(n + 11 = 12\\), and \\(\\gcd(6, 12) = 6\\). So, \\(d = 6\\) is possible. But we need to check if there are any other values of \\(n\\) that can give us \\(\\gcd(n + 5, n + 11) = 6\\). Since \\(n + 5\\) and \\(n + 11\\) differ 

In [38]:
x = response['post_distraction_response'].iloc[9]

In [37]:
def remove_boxed(s):
    if "\\boxed " in s:
        left = "\\boxed "
        assert s[:len(left)] == left
        return s[len(left):]

    left = "\\boxed{"

    assert s[:len(left)] == left
    assert s[-1] == "}"

    return s[len(left):-1]

In [46]:
remove_boxed(last_boxed_only_string(x))

'300'

In [50]:
response['solution'].iloc[10]

'If the number is divisible by $3$, the sum of the digits must be divisible by $3$.  So $a+b$ must be one more than a multiple of $3$, since $5$ is one less than a multiple of $3$.  We have several options:\n\nIf $a+b=1$, it must be $(1,0)$, one possibility.\n\nFor $a+b=4$, $a$ can be $1$ to $4$, for four possibilities.\n\nFor $a+b=7$, $a$ can be $1$ to $7$, so seven possibilities.\n\nFor $a+b=10$, anything from $(1,9)$ to $(9,1)$ works, so nine possibilities.\n\nFor $a+b=13$, the pairs range from $(4,9)$ to $(9,4)$, for six possibilities.\n\nIf $a+b=16$, we can have $(7,9)$, $(8,8)$, or $(9,7)$, so three possibilities.\n\n$a+b=19$ and up is not possible.\n\nSo the total number is\n\n$$1+4+7+9+6+3=\\boxed{30}$$'

In [54]:
response

,problem,level,type,solution,original_response,problem_distractor,distract_solution,distract_reasoning,actual_fraction,post_distraction_response,original_correct,distractor_correct
10,<|im_start|>system\nThe user asks a qusetion a...,Level 5,Geometry,The tetrahedron is shown below. In order to f...,"First, let's place the tetrahedron in a 3D coo...",<|im_start|>system\nThe user asks a qusetion a...,"Using the Euclidean Algorithm, we obtain \begi...","First, we need to find the greatest common div...",0.090909,The GCD of \(48n + 102\) and \(3n\) is the sam...,0.0,0.0
11,<|im_start|>system\nThe user asks a qusetion a...,Level 5,Geometry,The tetrahedron is shown below. In order to f...,"First, let's place the tetrahedron in a 3D coo...",<|im_start|>system\nThe user asks a qusetion a...,"Using the Euclidean Algorithm, we obtain \begi...","First, we need to find the greatest common div...",0.181818,"\[d = \gcd(48n + 102, 3n) = \gcd(102, 3n).\]\...",0.0,0.0
12,<|im_start|>system\nThe user asks a qusetion a...,Level 5,Geometry,The tetrahedron is shown below. In order to f...,"First, let's place the tetrahedron in a 3D coo...",<|im_start|>system\nThe user asks a qusetion a...,"Using the Euclidean Algorithm, we obtain \begi...","First, we need to find the greatest common div...",0.454545,"However, we need to check if there is a small...",0.0,0.0
13,<|im_start|>system\nThe user asks a qusetion a...,Level 5,Geometry,The tetrahedron is shown below. In order to f...,"First, let's place the tetrahedron in a 3D coo...",<|im_start|>system\nThe user asks a qusetion a...,"Using the Euclidean Algorithm, we obtain \begi...","First, we need to find the greatest common div...",0.727273,"However, this is not the maximum possible val...",0.0,0.0
17,<|im_start|>system\nThe user asks a qusetion a...,Level 5,Algebra,We are looking for some $x$ such that $f(x)=11...,Given the function \( f \) such that \( f(3) =...,<|im_start|>system\nThe user asks a qusetion a...,"Let $d = \gcd(n + 5, n + 11)$, so $d$ divides ...",Let's analyze the problem step by step. We nee...,0.428571,"Therefore, \(\gcd(n + 5, n + 11)\) can be 1.\...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
647,<|im_start|>system\nThe user asks a qusetion a...,Level 5,Geometry,We will calculate the area of the hexagon in t...,Let's denote the side length of the regular he...,<|im_start|>system\nThe user asks a qusetion a...,"Using the Euclidean Algorithm, we obtain \begi...","First, we need to find the greatest common div...",0.454545,"However, we need to check if there is a small...",0.0,0.0
648,<|im_start|>system\nThe user asks a qusetion a...,Level 5,Geometry,We will calculate the area of the hexagon in t...,Let's denote the side length of the regular he...,<|im_start|>system\nThe user asks a qusetion a...,"Using the Euclidean Algorithm, we obtain \begi...","First, we need to find the greatest common div...",0.727273,"However, this does not give us a consistent s...",0.0,0.0
650,<|im_start|>system\nThe user asks a qusetion a...,Level 4,Counting & Probability,[asy]\n\nLabel f;\n\nf.p=fontsize(6);\n\nxaxis...,"First, we need to determine the area of the re...",<|im_start|>system\nThe user asks a qusetion a...,"Let $d = \gcd(n + 5, n + 11)$, so $d$ divides ...",Let's analyze the problem step by step. We nee...,0.142857,We can use the properties of the greatest com...,0.0,0.0
652,<|im_start|>system\nThe user asks a qusetion a...,Level 4,Counting & Probability,[asy]\n\nLabel f;\n\nf.p=fontsize(6);\n\nxaxis...,"First, we need to determine the area of the re...",<|im_start|>system\nThe user asks a qusetion a...,"Let $d = \gcd(n + 5, n + 11)$, so $d$ divides ...",Let's analyze the problem step by step. We nee...,0.428571,"The probability that \(\gcd(n + 5, n + 11) = ...",0.0,0.0


In [ ]:
from math_verify import parse, verify

# Digit Corruption Results

In [5]:
def corruption_group_analysis (df: pd.DataFrame, percentile:str, stats: str):
    def group(pct):
        if pct <= .1:
            return .1
        elif pct <=.25:
            return .25
        elif pct <= 0.5:
            return .5
        elif pct <= .75:
            return .75
        else:
            return 1.0
    df['pctl_group'] = df[percentile].apply(group)
    return {"corruption percentage": df['pctl_group'].value_counts(), 
            "accuracy": df.groupby('pctl_group')[stats].mean()}

### Corrupt answer digit backwards

In [172]:
rl_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/Qwen2.5-Math-1.5B-DrGRPO-Zero_type=answer_digit_continuethinking=False.pickle")

In [173]:
corruption_group_analysis(rl_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 0.50    31
 1.00    27
 0.25    21
 0.75    19
 0.10    17
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.882353
 0.25    0.857143
 0.50    0.741935
 0.75    0.684211
 1.00    0.740741
 Name: still_correct, dtype: float64}

In [149]:
print(rl_df['post_corruption_response'][6])

 I started by recalling the definition of a complex number, which is \( z = x + yi \), where \( x \) and \( y \) are real numbers, and \( i \) is the imaginary unit. The conjugate of \( z \), denoted \( \bar{z} \), is \( x - yi \). 

I then substituted \( z = x + yi \) and \( \bar{z} = x - yi \) into the given equation \( 3z + 5i\bar{z} = 1 - 8i \). Expanding both terms, I got \( 3x + 3yi + 5ix - 5y = (3x + 5y) + (3y + 5x)i \). 

Setting the real and imaginary parts equal to the corresponding parts on the right-hand side, I obtained the system of linear equations: \( 3x + 5y = 1 \) and \( 5x + 3y = -8 \). 

To solve this system, I used the method of elimination. I multiplied the first equation by 5 and the second equation by 3 to align the coefficients of \( x \), resulting in the equations \( 12x + 16y = 5 \) and \( 12x + 9y = -25 \). Subtracting the second equation from the first, I found \( 7y = 28 \), leading to \( y = 5 \). Substituting \( y = 5 \) back into the first equation, I 

In [91]:
rl_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/R1-Distill-Qwen-1.5B_type=answer_digit_continuethinking=False.pickle")

In [92]:
corruption_group_analysis(rl_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 0.50    298
 0.25    290
 1.00    153
 0.75    151
 0.10    143
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.335664
 0.25    0.082759
 0.50    0.097315
 0.75    0.079470
 1.00    0.104575
 Name: still_correct, dtype: float64}

### Corrupt all digits forwards

In [174]:
sft_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/Qwen2.5-Math-1.5B-DrGRPO-Zero_type=midway_continuethinking=True.pickle")

In [175]:
corruption_group_analysis(sft_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 1.00    137
 0.50     71
 0.75     44
 0.25     27
 0.10      2
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.500000
 0.25    0.407407
 0.50    0.183099
 0.75    0.136364
 1.00    0.138686
 Name: still_correct, dtype: float64}

In [152]:
rl_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/Qwen2.5-Math-1.5B-sky-math8k-r1-continue-sft64steps-rl300steps_type=midway_continuethinking=True.pickle")

In [153]:
corruption_group_analysis(rl_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 1.00    106
 0.50     79
 0.75     65
 0.25     43
 0.10      4
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.250000
 0.25    0.302326
 0.50    0.113924
 0.75    0.107692
 1.00    0.216981
 Name: still_correct, dtype: float64}

# Distraction

## Full distraction

In [3]:
root = '/share/goyal/lio/reasoning/eval/hendrycks_math_qwen32b/distract_reasoning'
os.listdir(root)

['full_distract_qwen32b-200sample.pickle',
 'full_distract_r1-qwen32b-200sample.pickle']

In [6]:
df = pd.read_pickle(os.path.join(root, 'full_distract_qwen32b-200sample.pickle'))

print("accuracy according to original solution: \n", corruption_group_analysis(df, 'actual_fraction', 'original_correct'))
print('*' * 50, '\n\n')
print("accuracy according to distractor solution: \n", corruption_group_analysis(df, 'actual_fraction', 'distractor_correct'))

accuracy according to original solution: 
 {'corruption percentage': pctl_group
0.25    167
0.75    167
0.50    167
1.00    167
0.10    143
Name: count, dtype: int64, 'accuracy': pctl_group
0.10    0.545455
0.25    0.485030
0.50    0.431138
0.75    0.257485
1.00    0.035928
Name: original_correct, dtype: float64}
************************************************** 


accuracy according to distractor solution: 
 {'corruption percentage': pctl_group
0.25    167
0.75    167
0.50    167
1.00    167
0.10    143
Name: count, dtype: int64, 'accuracy': pctl_group
0.10    0.265734
0.25    0.287425
0.50    0.329341
0.75    0.694611
1.00    0.976048
Name: distractor_correct, dtype: float64}


In [8]:
df = pd.read_pickle(os.path.join(root, 'full_distract_r1-qwen32b-200sample.pickle'))

print("accuracy according to original solution: \n", corruption_group_analysis(df, 'actual_fraction', 'original_correct'))
print('*' * 50, '\n\n')
print("accuracy according to distractor solution: \n", corruption_group_analysis(df, 'actual_fraction', 'distractor_correct'))

accuracy according to original solution: 
 {'corruption percentage': pctl_group
0.25    168
0.75    168
0.50    168
1.00    168
0.10    131
Name: count, dtype: int64, 'accuracy': pctl_group
0.10    0.061069
0.25    0.047619
0.50    0.059524
0.75    0.041667
1.00    0.017857
Name: original_correct, dtype: float64}
************************************************** 


accuracy according to distractor solution: 
 {'corruption percentage': pctl_group
0.25    168
0.75    168
0.50    168
1.00    168
0.10    131
Name: count, dtype: int64, 'accuracy': pctl_group
0.10    0.786260
0.25    0.815476
0.50    0.827381
0.75    0.869048
1.00    0.988095
Name: distractor_correct, dtype: float64}


In [13]:
df["post_distraction_response"][3]

" Then, count them.\n\nBut wait, there's a property related to digit sums and divisibility. Specifically, a number is divisible by 9 if and only if the sum of its digits is divisible by 9. So, any number with a digit sum of 9 is divisible by 9. Therefore, unless the number itself is 9, which isn't prime, all other numbers with digit sum 9 are composite because they're divisible by 9.\n\nWait, that's a crucial point. So, if a number has a digit sum of 9, it's divisible by 9, which means it's composite unless the number is 9 itself. But 9 is not prime because it's divisible by 3. So, does that mean that there are no prime numbers with digit sum 9?\n\nWait, hold on. Let me think again. If a number's digits add up to 9, it's divisible by 9, right? So, the number itself must be a multiple of 9. Therefore, the only prime multiple of 9 is 3, but 3 is not a multiple of 9. Wait, actually, 9 is 3 squared, so it's composite. So, any number divisible by 9 is either 9 itself or a composite number.\

In [17]:
df["distract_reasoning"][0]

'Alright, so I came across this probability problem, and it seems a bit tricky at first glance, but I think I can break it down step by step. Let me try to work through it.\n\nThe problem is: A number is chosen uniformly at random from among the positive integers less than \\(10^8\\). Given that the sum of the digits of the number is 9, what is the probability that the number is prime?'

In [18]:
df["post_distraction_response"][0]

" Hmm, okay. So, we're dealing with numbers less than 100 million, which is \\(10^8\\). That means we're considering all positive integers from 1 up to 99,999,999. \n\nFirst, I need to understand the problem clearly. We're choosing a number uniformly at random, so every number in that range has an equal chance of being selected. However, we have a condition: the sum of the digits of the number is 9. Given that condition, we need to find the probability that the number is prime.\n\nProbability is calculated as the number of favorable outcomes divided by the total number of possible outcomes. In this case, the favorable outcomes are the prime numbers whose digits sum to 9, and the total possible outcomes are all numbers less than \\(10^8\\) whose digits sum to 9.\n\nSo, my first task is to figure out how many numbers less than \\(10^8\\) have digits that add up to 9. Then, I need to determine how many of those numbers are prime. The probability will be the ratio of these two quantities.\